In [17]:
import pandas as pd
from pydataxm import *
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer, Date, Float  # Importar tipos necesarios
import  os
import psycopg2

In [18]:
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

In [19]:
#Definir periodo de los datos
today = dt.date.today()
month_ago = today - dt.timedelta(days=360) # Approximation of a month

In [20]:
#Conexion a la API
objetoAPI = pydataxm.ReadDB()

**AGENTES**

In [21]:
#Descargar los datos
df_Agentes = objetoAPI.request_data(
                        "ListadoAgentes",           #Se indica el nombre de la métrica tal como se llama en el campo metricId
                        "Sistema",               #Se indica el nombre de la entidad tal como se llama en el campo Entity
                        month_ago,     #Corresponde a la fecha inicial de la consulta
                        today)


df_Agentes.rename(columns={'Values_Code': 'pkAgente',
                           'Id':'IdAgentes',
                          'Values_Activity':'Actividad',
                          'Values_State':'Estado'}, inplace=True)
df_Agentes=df_Agentes[['pkAgente','Actividad','Estado']]
df_Agentes.head()


df_Agentes["fecha_actualizacion"] = today
df_Agentes.head()

,pkAgente,Actividad,Estado,fecha_actualizacion
0,ESNC,COMERCIALIZACIÓN,OPERACION,2025-06-02
1,SEJC,COMERCIALIZACIÓN,OPERACION,2025-06-02
2,OTAG,GENERACIÓN,OPERACION,2025-06-02
3,DUCG,GENERACIÓN,OPERACION,2025-06-02
4,ITLC,COMERCIALIZACIÓN,OPERACION,2025-06-02


In [22]:
# Insertar datos en PostgreSQL (modo 'append' para no borrar datos existentes)
with engine.begin() as conn:  # <- Se hace COMMIT automático al final
    df_Agentes.to_sql(
    name='agentes',          # Nombre de la tabla (sin esquema aquí)
    con=engine,
    schema='tablasxm_dev',       # Esquema especificado aquí
    if_exists='replace',          # Opción: 'append', 'replace' o 'fail'
    index=True,                 # Incluir el índice del DataFrame
    dtype={
         'pkagente': String(),           # INT4 en PostgreSQL
        'Actividad': String(),                  # DATE
        'Estado': String(),               # INT4
        'fecha_actualizacion': Date()
    })

print("Datos insertados correctamente.")

Datos insertados correctamente.


In [23]:
print(len(df_Agentes))

650


**RECURSOS**

In [24]:
df_Recursos = objetoAPI.request_data(
                        "ListadoRecursos",           #Se indica el nombre de la métrica tal como se llama en el campo metricId
                        "Sistema",               #Se indica el nombre de la entidad tal como se llama en el campo Entity
                        month_ago,     #Corresponde a la fecha inicial de la consulta
                        today)

df_Recursos.rename(columns={
    'Values_CompanyCode': 'pkAgente',
    'Values_Code':'pkRecurso',
    'Id':'IdListadoRecursos',
    'Values_Name':'NombreRecurso',
    'Values_Type':'Tipo_Recurso'}, inplace=True)
df_Recursos=df_Recursos[['pkRecurso','pkAgente','NombreRecurso','Tipo_Recurso']]
df_Recursos["fecha_actualizacion"] = today
df_Recursos.head()

,pkRecurso,pkAgente,NombreRecurso,Tipo_Recurso,fecha_actualizacion
0,2QBW,ISGG,EL POPAL,HIDRAULICA,2025-06-02
1,2QEK,ENDG,SALTO II,HIDRAULICA,2025-06-02
2,2QRL,HZEG,LA REBUSCA,HIDRAULICA,2025-06-02
3,2QV2,EPSG,BAJO TULUA,HIDRAULICA,2025-06-02
4,2R22,ENDG,LAGUNETA,HIDRAULICA,2025-06-02


In [25]:
# Insertar datos en PostgreSQL (modo 'append' para no borrar datos existentes)
with engine.begin() as conn:  # <- Se hace COMMIT automático al final
    df_Recursos.to_sql(
    name='recursos',          # Nombre de la tabla (sin esquema aquí)
    con=engine,
    schema='tablasxm_dev',       # Esquema especificado aquí
    if_exists='replace',          # Opción: 'append', 'replace' o 'fail'
    index=True,                 # Incluir el índice del DataFrame
    dtype={
          'pkrecurso': String(),      # TEXT en PostgreSQL (equivale a String de SQLAlchemy)
        'pkagente': String(),           # INT4 en PostgreSQL
        'nombre_recurso': Date(),                  # DATE
        'tipo_recurso': Integer(),               # INT4
        'fecha_actualizacion': Date()
    })



print("Datos insertados correctamente.")

Datos insertados correctamente.


In [27]:
print(len(df_Recursos))

1013
